In [13]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
import pickle
from sklearn import naive_bayes, metrics, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

<h2>lấy dữ liệu đã được xử lý</h2>
<p>dữ liệu phục vụ cho mô hình là một csr matrix, hay nói cách khác là một tập hợp gồm n phần tử, mỗi phần tử là một vector p chiều</p>
<p>mỗi chiều của vector là một từ trong tiếng việt, bao gồm cả từ đơn và từ ghép</p>
<p>file .pkl dùng để lưu lại các object(list, tuple ...) trong python</p>
<p><strong>count_vect_data lưu lại một ma trận có 40000 cột và số hàng tương ứng với số văn bản, mỗi phần tử là số lần xuất hiện của từ trong văn bản</strong><p>
<p><strong>tfidf như cont_vect, nhưng giá trị phần tử là tần suất thay vì số lần xuất hiện</strong></p>
<p><strong>tfidf_svd là tfidf nhưng đã giảm số chiều(số cột)</strong></p>

In [5]:
X_data_count = pickle.load(open("count_vect_data.pkl", "rb"))
X_test_count = pickle.load(open("count_vect_test.pkl", "rb"))
X_data_tfidf_svd = pickle.load(open("X_data_tfidf_svd.pkl", "rb"))
X_test_tfidf_svd = pickle.load(open("X_test_tfidf_svd.pkl", "rb"))
y_data = pickle.load(open("raw_data/y_data.pkl", "rb"))
y_test = pickle.load(open("raw_data/y_test.pkl", "rb"))
X_data = pickle.load(open("raw_data/X_data.pkl", "rb"))
X_test = pickle.load(open("raw_data/X_test.pkl", "rb"))
X_data_tfidf = pickle.load(open("X_data_tfidf.pkl", "rb"))

<h2>đánh nhãn cho các lớp, do model hoạt động dựa trên dữ liệu số</h2>


In [6]:
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

encoder.classes_ # kết quả: array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
                 #                 'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
                 #                 'Vi tinh'], dtype='<U16')

#cell nay bo cx dc, cac thuat toan van chay binh thuong

array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
       'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
       'Vi tinh'], dtype='<U16')

<h2>build hàm huấn luyện chung để cho các model để thuận tiện thay đổi model</h2>

In [7]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)

        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)

 
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))

<h1>Tiến hành build các model</h1>

<h2>1, Naive-bayes</h2>

In [8]:
naivBayesType = naive_bayes.MultinomialNB()
train_model(naivBayesType, X_data_count, y_data, X_test_count, y_test, is_neuralnet=False)

Validation accuracy:  0.9164490861618799
Test accuracy:  0.7852889667250438


<h2>2, SVM</h2>
<p>svm và knn test dữ liệu có số chiều nhỏ<p>

In [9]:
from sklearn import svm
svmModel = svm.SVC()
train_model(svmModel, X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

Validation accuracy:  0.9295039164490861
Test accuracy:  0.7957968476357268


<h2>3, KNN</h2>


In [10]:
from sklearn import neighbors
NB_model = neighbors.KNeighborsClassifier()
train_model(NB_model, X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

Validation accuracy:  0.8459530026109661
Test accuracy:  0.6612959719789843


<h2> sử dụng model </h2>

<p>tạo mẫu</p>

các tư thế xử lý dữ liệu

In [22]:
from sklearn.decomposition import TruncatedSVD
#chuyển data về kiểu tfidf
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
#chuyển data về kiểu giảm số chiều
svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)
#chuyển data về kiểu đếm số lần xuất hiện 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)



In [ ]:
test = ["Số ca Covid-19 ghi nhận mới trên cả nước đang có dấu hiệu gia tăng trở lại. Theo thống kê của Bộ Y tế, trong ngày 10/4, cả nước ghi nhận 113 ca Covid-19. Trong đó, Hà Nội vẫn là địa phương đứng đầu cả nước về số ca Covid-19 mới."]
example_count_type = count_vect.transform(test)
example = tfidf_vect.transform(test)
example_truncated = svd.transform(example)

In [17]:
print(len(example_truncated))

1


dự đoán

In [23]:
print("kết quả của naive-bayes model: {}".format(naivBayesType.predict(example_count_type)))
print("kết quả của SVM model: {}".format(svmModel.predict(example_truncated)))
print("kết quả của KNN model: {}".format(NB_model.predict(example_truncated)))

kết quả của naive-bayes model: ['Suc khoe']
kết quả của SVM model: ['Suc khoe']
kết quả của KNN model: ['Vi tinh']
